In [1]:
# after kernel restart, always execute this line
%matplotlib inline
import sys
sys.path.insert(0, 'path/to/everest-basecamp')

ML-developer part
---------------------------
Below is the a pytorch code to develop a small DNN inference application using already trained coefficients. Also, we use parts of the training data set to better calibrate the weights for the quantized domain.

In [2]:
path_prefix = './tpred_v7_20230417/'

In [3]:
import torch
import torch.nn as nn

import numpy as np


class TPNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.dense1 = nn.Linear(17, 16)
        self.dense2 = nn.Linear(16, 8)
        self.dense3 = nn.Linear(8, 4)

        self.L1 = self.dense1
        self.L2 = self.dense2
        self.L3 = self.dense3
        self.all_weights = [self.dense1.weight, self.dense1.bias, self.dense2.weight, self.dense2.bias,
                            self.dense3.weight, self.dense3.bias]

    def forward(self, x):
        x = self.relu(self.dense1(x))
        x = self.relu(self.dense2(x))
        x = self.dense3(x)
        return x


def _tf_to_torch_param(x: np.ndarray):
    ret = nn.parameter.Parameter(torch.tensor(x.T).type(torch.FloatTensor))
    return ret


def load_from_pkl_files(dnn: nn.Module, used_rid=0):
    # from tf2_174_infer_sequential_M10_2.py
    import pickle
    import glob
    import re

    NI = 17  # kernel vector input size
    L1 = 16  # layer 1 internal size
    L2 = 8  # layer 2 internal size
    NO = 4  # kernel vector output size
    M = 10  # number of models

    # NN coefficients data read-only storage
    W1 = np.zeros((M, NI, L1))
    B1 = np.zeros((M, L1))
    W2 = np.zeros((M, L1, L2))
    B2 = np.zeros((M, L2))
    W3 = np.zeros((M, L2, NO))
    B3 = np.zeros((M, NO))

    # reading all coefficient files _coeffs_xxx.pkl (pickle)
    cflist = [f for f in glob.glob(path_prefix+"*.pkl")]
    for filename in cflist:
        match = re.search('_coeffs_(.+?).pkl', filename)
        rid = 0
        if match:
            rid = int(match.group(1))
        if rid < 0 or rid >= M:
            continue  # ignore models oo range
        # print("reading dataset", rid)

        with open(filename, 'rb') as f:
            coeffs = pickle.load(f)
            W1[rid] = np.array(coeffs[0])
            B1[rid] = np.array(coeffs[1])
            W2[rid] = np.array(coeffs[2])
            B2[rid] = np.array(coeffs[3])
            W3[rid] = np.array(coeffs[4])
            B3[rid] = np.array(coeffs[5])

    dnn.L1.weight = _tf_to_torch_param(W1[used_rid])
    dnn.L1.bias = _tf_to_torch_param(B1[used_rid])
    dnn.L2.weight = _tf_to_torch_param(W2[used_rid])
    dnn.L2.bias = _tf_to_torch_param(B2[used_rid])
    dnn.L3.weight = _tf_to_torch_param(W3[used_rid])
    dnn.L3.bias = _tf_to_torch_param(B3[used_rid])

    return dnn


def read_example_vectors():
    N = 1000
    M = 10  # number of models
    # data vector inputs
    import pandas as pd
    d = pd.read_csv(path_prefix + "_traffic_dataset_20230426.csv")
    d = d.to_numpy()
    # sel = d[0:N, 0]
    # sel = sel.astype(int)
    # sel = np.array([x % M for x in sel])  # temporary hack
    x = d[0:N, 1:18]
    return x



/home/ngl/gitrepos/EVEREST/everest-basecamp/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

torch_store_path = './etp_01.pt'
dnn = TPNN()
load_from_pkl_files(dnn)

print(f'Storing DNN to {torch_store_path}')
# export weights as state dict
torch.save(dnn.state_dict(), f"{torch_store_path}_state_dict")
# export complete model as torch script
model_scripted = torch.jit.script(dnn)
model_scripted.save(torch_store_path)



Storing DNN to ./etp_01.pt


In [5]:
x = read_example_vectors()
numpy_store_path = './traffic_data_2023-04-26.npy'
np.save(numpy_store_path, x)

The next cell is just there to show also the pytorch model works...

In [6]:

used_road_id = 0
M = 10  # still ocrrect?
N = 1000

Y_out = []
with torch.no_grad():
    for index in range(N):
        if index % 1000 == 0:
            print("inference", index)
        road_id = used_road_id
        vin = torch.tensor(x[index]).type(torch.FloatTensor)
        y_pred = dnn(vin)
        y_numpy = y_pred.numpy()
        Y_out.append((road_id, y_numpy))

for item in Y_out[0:10]:
    print(item)

inference 0
(0, array([30.545656, 40.80023 , 34.346386, 42.262363], dtype=float32))
(0, array([25.729465, 24.704857, 16.780256, 26.248438], dtype=float32))
(0, array([35.45301 , 34.960674, 34.22544 , 43.827374], dtype=float32))
(0, array([4.0733585, 2.9853544, 2.1709018, 2.6423662], dtype=float32))
(0, array([37.432034, 47.356346, 43.17812 , 49.02317 ], dtype=float32))
(0, array([28.012989, 24.696459, 21.040972, 27.595472], dtype=float32))
(0, array([41.50566, 41.57171, 34.12154, 52.50362], dtype=float32))
(0, array([15.028785 , 16.173925 , 14.4315815, 16.47795  ], dtype=float32))
(0, array([37.151417, 43.334423, 38.131054, 52.717567], dtype=float32))
(0, array([25.99692 , 27.607416, 21.113655, 28.623783], dtype=float32))


In [7]:
! ls -l | grep *.pt

-rw-rw-r--. 1 ngl ngl   13553 Jan 29 18:50 etp_01.pt
-rw-rw-r--. 1 ngl ngl    4111 Jan 29 18:50 etp_01.pt_state_dict


Now, we start using **Everest basecamp**

In [2]:
from ebc import basecamp


# batch_size = 10000
batch_size = 1
precision = 8

emli = basecamp.ml_inference
emli.set_constraints(app_name="EVEREST traffic prediction v1",
                     onnx_input_name='x',
                     input_shape=[batch_size, 17],
                     input_size_t=precision,
                     quantization='int8',  # that's new
                     batch_size=batch_size,
                     target_throughput=1,  # batch_size inferences per second
                     arch_gen_strategy='throughput'
                     )
emli.set_output_path('traffic_prediction/build_dirs/etp_v1')

emli.set_model_path("torchscript", './etp_01.pt')

emli.set_calibration_data_path('./traffic_data_2023-04-26.npy')

In [ ]:
emli.disable_build = True  # or define environemnt_path DOSA_cFBuild1_used_dcps_path
emli.compile()